# Quack Quack - Creating the DuckDB

---

**Ducking the Data with a Database**

As this project will involve many data transformations; engineered features; and iterative modeling; I need an orderly, robust system to handle all of the data, models, etc. without creating too much complexity in the repository. Instead of creating separate files for each version of the data, I decided that I need to create a small database to store the information  effectively and efficiently. Before I can create the database, I need data!

**Hatching the Plan**

I obtained my source data from the article referenced in the `README.md` file. The source data comes in the form of two separate CSV files, which are both sizeable and take a while to load into a dataframe (or in this case, a database). To reduce the size and increase read/write times, I will convert the original source files from CSVs to parquet files. Then, I will take the raw reservation data and use it to create the first table within the database.

---

In [1]:
import duckdb
import glob
import os
import pandas as pd
import uuid

# Convert Source CSVs to Parquet

---

The following code loops through this repository's `/data/` directory; searches for the source CSVs; converts each of them to a parquet file; and then deletes the CSV.

---

In [2]:
# Define the directory containing the CSV files
directory = './data/'

# Pattern match for files named 'h1.csv' or 'h2.csv'
file_patterns = [os.path.join(directory, 'h1.csv'), os.path.join(directory, 'h2.csv')]

# Initialize a list to hold the matched file paths
csv_files = []

# Loop through the patterns and extend the list with found files
for pattern in file_patterns:
    csv_files.extend(glob.glob(pattern))

# Check if no files were found
if not csv_files:
    print("No matching filepaths found. Stopping execution.")
else:
    # Loop through each found CSV file
    for csv_file in csv_files:
        try:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(csv_file)
            
            df['HotelNumber'] = csv_file[1:2]
            
            # Define the Parquet file path (same name as the CSV file but with .parquet extension)
            parquet_file = csv_file.replace('.csv', '.parquet')
            
            # Convert the DataFrame to a Parquet file
            df.to_parquet(parquet_file)

            # If the conversion was successful, remove the CSV file
            os.remove(csv_file)
            print(f"Successfully converted and removed {csv_file}")
        except Exception as e:
            print(f"Error converting {csv_file}: {e}")

    print("Conversion completed.")

No matching filepaths found. Stopping execution.


# Generate and Append UUIDs

Since the source data was anonymized, there are no unique identifiers for each reservation. To support database joins and relationships between tables, I will add columns for both a UUID and the source hotel number to differentiate the reservations and preserve the unique details of each hotel.

In [3]:
def generate_uuid_list(df):

    # Generate a UUID for each row and save to a list
    uuid_list = [str(uuid.uuid4()) for _ in range(len(df))]
    
    return uuid_list
    

def extract_hotel_number(filepath):
    
    # Extract the base filename without the extension
    base_filename = os.path.basename(filepath).split('.')[0]
    
    # Extract the hotel number (assuming the format is always H<number>)
    hotel_number = base_filename[1:]
    
    return int(hotel_number)


def add_hotel_number_to_dataframe(input_filepath,
                                  output_filepath = None,
                                  col_hotelnum = 'HotelNumber',
                                  col_id = 'UUID',
                                  save_to_parquet = True,
                                  engine = 'pyarrow',
                                  compression = 'snappy'):
    
    # Read the Parquet file into a DataFrame
    df = pd.read_parquet(input_filepath)
    
    # Extract the hotel number from the filename and append to dataframe
    df[col_hotelnum] = extract_hotel_number(input_filepath)
    
    # Generate and append UUIDs for each reservation
    df[col_id] = generate_uuid_list(df)
    
    if save_to_parquet == True and output_filepath != None:
        df.to_parquet(output_filepath, engine, compression)
    else:
        pass
    
    return df

In [4]:
input_files = ['./data/H1.parquet', './data/H2.parquet']
output_files = ['./data/H1_with_uuid.parquet', './data/H2_with_uuid.parquet']

save = True

for input, output in zip(input_files, output_files):
    df = add_hotel_number_to_dataframe(input,output, save_to_parquet = save)
    display(df.head())
    del df

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,HotelNumber,UUID
0,0,342,2015,July,27,1,0,0,2,0,...,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01,1,ae05b69e-fcce-4b16-99d8-cf26faf9c4e6
1,0,737,2015,July,27,1,0,0,2,0,...,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01,1,473a8d2f-6d16-41f1-a22f-e77f2fea5b7a
2,0,7,2015,July,27,1,0,1,1,0,...,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02,1,7ff91b07-290f-4061-a823-377c47aadc08
3,0,13,2015,July,27,1,0,1,1,0,...,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02,1,3342411f-c149-4d0a-a400-ac21e757eb21
4,0,14,2015,July,27,1,0,2,2,0,...,NULL,0,Transient,98.0,0,1,Check-Out,2015-07-03,1,6e81bc1a-c90b-46d5-b90c-d3509dc939c8


,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,HotelNumber,UUID
0,0,6,2015,July,27,1,0,2,1,0.0,...,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-03,2,8dd3419b-ca30-40cf-84c8-cc0e5a1326a4
1,1,88,2015,July,27,1,0,4,2,0.0,...,NULL,0,Transient,76.5,0,1,Canceled,2015-07-01,2,e86b8c61-c1e8-49e5-bc81-23f3722a3302
2,1,65,2015,July,27,1,0,4,1,0.0,...,NULL,0,Transient,68.0,0,1,Canceled,2015-04-30,2,d903fd4f-cc32-4a55-94d9-b3d04797a3e0
3,1,92,2015,July,27,1,2,4,2,0.0,...,NULL,0,Transient,76.5,0,2,Canceled,2015-06-23,2,963ee5c8-7c7a-4984-880f-f4ad7c5fd11d
4,1,100,2015,July,27,2,0,2,2,0.0,...,NULL,0,Transient,76.5,0,1,Canceled,2015-04-02,2,dd5d79d2-c91b-4ff3-b64a-de2012a426f0


In [5]:
# ## Loop through the source data; generate and append UUIDs; append the hotel number;
# ## then save the new results as a new parquet file.

# input_files = ['./data/H1.parquet', './data/h2.parquet']
# output_files = ['./data/H1_with_uuid.parquet', './data/H2_with_uuid.parquet']

# # Process each file
# for input_file, output_file in zip(input_files, output_files):
#     add_uuids_to_parquet(input_file, output_file)
# # List of Parquet file paths
# file_paths = ['./data/H1_with_uuid.parquet', './data/H2_with_uuid.parquet']

# for file in file_paths:
#     print(file[8])
    
#     temp_df = pd.read_parquet(file)
#     temp_df['HotelNumber'] = file[8]
#     temp_df.to_parquet(file)
# for file in file_paths:
#     temp_df = pd.read_parquet(file)
#     display(temp_df[['uuid', 'HotelNumber']].head(10))
#     del temp_df

# Convert Updated Parquets to DuckDB

---

After converting the source CSVs to parquet form, I will now create the database to be used in the rest of the project pipeline.

---

In [6]:
# List of Parquet file paths
file_paths = ['./data/H1_with_uuid.parquet', './data/H2_with_uuid.parquet']

# Path to the DuckDB database file
db_path = './data/hotel_reservations.duckdb'

# Check if the database file exists and remove it if it does
if os.path.exists(db_path):
    os.remove(db_path)

# Initialize connection to DuckDB
conn = duckdb.connect(database=db_path, read_only=False)

# Use the first file to create the table
conn.execute(f"CREATE TABLE source_data AS SELECT * FROM '{file_paths[0]}'")

# For subsequent files, append data to the existing table
for file_path in file_paths[1:]:  # Start from the second item
    conn.execute(f"INSERT INTO source_data SELECT * FROM '{file_path}'")
    
## Confirm successful creation of database and table(s)
display(conn.execute('SELECT * FROM source_data LIMIT 10').df())

conn.close()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,HotelNumber,UUID
0,0,342,2015,July,27,1,0,0,2,0,...,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01,1,ae05b69e-fcce-4b16-99d8-cf26faf9c4e6
1,0,737,2015,July,27,1,0,0,2,0,...,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01,1,473a8d2f-6d16-41f1-a22f-e77f2fea5b7a
2,0,7,2015,July,27,1,0,1,1,0,...,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02,1,7ff91b07-290f-4061-a823-377c47aadc08
3,0,13,2015,July,27,1,0,1,1,0,...,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02,1,3342411f-c149-4d0a-a400-ac21e757eb21
4,0,14,2015,July,27,1,0,2,2,0,...,NULL,0,Transient,98.0,0,1,Check-Out,2015-07-03,1,6e81bc1a-c90b-46d5-b90c-d3509dc939c8
5,0,14,2015,July,27,1,0,2,2,0,...,NULL,0,Transient,98.0,0,1,Check-Out,2015-07-03,1,25b31fab-f2ec-4aa2-969f-8f3594d7e030
6,0,0,2015,July,27,1,0,2,2,0,...,NULL,0,Transient,107.0,0,0,Check-Out,2015-07-03,1,e929f7f9-4134-4bfb-96ad-a46677684c4c
7,0,9,2015,July,27,1,0,2,2,0,...,NULL,0,Transient,103.0,0,1,Check-Out,2015-07-03,1,154adac9-0d7a-41b8-81c2-cf54d99a40ee
8,1,85,2015,July,27,1,0,3,2,0,...,NULL,0,Transient,82.0,0,1,Canceled,2015-05-06,1,5de368f9-5b64-49b9-a477-352cfc3fd3c6
9,1,75,2015,July,27,1,0,3,2,0,...,NULL,0,Transient,105.5,0,0,Canceled,2015-04-22,1,f41e5c91-cdff-4732-9254-2effdf0f45e1


In [7]:
for file in file_paths:

    # Remove intermediate parquet files
    if os.path.exists(file):
        os.remove(file)